In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from glob import glob
from zipfile import ZipFile
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
%matplotlib inline

In [ ]:
path = r"H:\archive.zip"

destination = r"H:\archive.zip" # Extract train and test folders here

In [ ]:
# Unzipping train and test folders
for folder in os.listdir(path = path):
    if folder.split(".")[1] == "zip": 
        with ZipFile(path + folder, 'r') as zip:
            zip.extractall(destination)
            print(f'{folder} extracted done!')

In [ ]:
for folder in os.listdir(destination):
    files = glob(destination + folder + "/*.jpg")
    print(f'({folder}): {len(files)} image')

In [ ]:
# Shape of training images

train_images_size = []

files = glob(destination + "train/" + "/*.jpg")
for file in files:
    img = cv2.imread(file)
    train_images_size.append(img.shape)
        
pd.Series(train_images_size).value_counts()

In [ ]:
# Shape of test images

test_images_size = []

files = glob(destination + "test1" + "/*.jpg")
for file in files:
    img = cv2.imread(file)
    train_images_size.append(img.shape)
        
pd.Series(train_images_size).value_counts()

In [ ]:
categorical = {"cat":0, "dog":1} 
image_size  = 150

In [ ]:
def getKey(myValue):
    for key, value in categorical.items():
        if myValue == value:
            return key

In [ ]:
X = []
y = []

files = glob(destination + "train/*.jpg")
for file in files:
        
    if "dog" in file:
        y.append(1)
    else:
        y.append(0)
        
    img = cv2.imread(file)
    new_train_size = cv2.resize(img, (image_size, image_size))
    X.append(new_train_size)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
print(f"X_train: {len(X_train)} , y_train: {len(y_train)}")
print(f"X_test: {len(X_test)} , y_test: {len(y_test)}")

In [ ]:
plt.figure(figsize=(10,10))
for n , i in enumerate(list(np.random.randint(0,len(X_train),9))) : 
    plt.subplot(3,3,n+1)
    plt.imshow(X_train[i])   
    plt.axis('off')
    plt.title(getKey(y_train[i]))
    

In [ ]:
pred = []

files = glob(destination + "test1/" + "/*.jpg")
for file in files:
    img = cv2.imread(file)
    new_test_size = cv2.resize(img, (image_size, image_size))
    pred.append(new_train_size)

In [ ]:
print(f"pred: {len(pred)}")

In [ ]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test  = np.array(X_test)
y_test  = np.array(y_test)
X_pred  = np.array(pred)

In [ ]:
print(f'X_train shape  is {X_train.shape}')
print(f'y_train shape  is {y_train.shape} \n')
print(f'X_test  shape  is {X_test.shape}')
print(f'y_test  shape  is {y_test.shape} \n')
print(f'X_pred  shape  is {X_pred.shape}')

In [ ]:
Model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(128,kernel_size=(3,3),activation='relu',input_shape=(image_size,image_size,3)),
        tf.keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
        #tf.keras.layers.AveragePooling2D(3,3),
        tf.keras.layers.MaxPool2D(4,4),
    
        tf.keras.layers.Conv2D(128,kernel_size=(3,3),activation='relu'),    
        tf.keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
        #tf.keras.layers.AveragePooling2D(3,3),
        tf.keras.layers.MaxPool2D(4,4),
    
        tf.keras.layers.Flatten() ,    
        tf.keras.layers.Dense(512,activation='relu') ,    
        #tf.keras.layers.Dense(64,activation='relu') ,
        #tf.keras.layers.Dense(16,activation='relu') ,
        tf.keras.layers.Dropout(rate=0.3),
        tf.keras.layers.Dense(1, activation='sigmoid'),    
        ])


In [ ]:
Model.compile(optimizer =tf.keras.optimizers.Adam(learning_rate=0.0001),
                   loss='binary_crossentropy',
                   metrics=['accuracy'])

In [ ]:
history = Model.fit(X_train, y_train, epochs=50)

In [ ]:
pd.DataFrame(history.history).plot()
plt.ylabel("Value")
plt.xlabel("Epochs");

In [ ]:
Loss, Accuracy = Model.evaluate(X_test, y_test)

print('Loss is: ', Loss)
print('Accuracy is: ', Accuracy)